In [1]:
import pymongo
from pprint import pprint

## Database Connection

In [2]:
db = pymongo.MongoClient()

## Inspect

In [3]:
db.database_names()

['local', 'test']

In [4]:
db['test'].collection_names()

['zips']

In [5]:
# db.collection.findOne()
pprint(
    db['test']['zips'].find_one()
)

{'_id': '01007',
 'city': 'BELCHERTOWN',
 'loc': [-72.410953, 42.275103],
 'pop': 10579,
 'state': 'MA'}


In [6]:
# db.collection.count()
db['test']['zips'].count()

29353

## Aggregation Pipeline

### stage operators I

In [7]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:5])

[{'_id': '10001',
  'city': 'NEW YORK',
  'loc': [-73.996705, 40.74838],
  'pop': 18913,
  'state': 'NY'},
 {'_id': '10002',
  'city': 'NEW YORK',
  'loc': [-73.987681, 40.715231],
  'pop': 84143,
  'state': 'NY'},
 {'_id': '10003',
  'city': 'NEW YORK',
  'loc': [-73.989223, 40.731253],
  'pop': 51224,
  'state': 'NY'},
 {'_id': '10005',
  'city': 'NEW YORK',
  'loc': [-74.008344, 40.705649],
  'pop': 202,
  'state': 'NY'},
 {'_id': '10006',
  'city': 'NEW YORK',
  'loc': [-74.013474, 40.708451],
  'pop': 119,
  'state': 'NY'}]


In [8]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"city": 1, "_id": 0}}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:3])

[{'city': 'NEW YORK'}, {'city': 'NEW YORK'}, {'city': 'NEW YORK'}]


### stage operators II

In [9]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"town": "$city", "_id": 1}},
    {"$sort": {'_id': -1}}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:3])

[{'_id': '10280', 'town': 'NEW YORK'},
 {'_id': '10128', 'town': 'NEW YORK'},
 {'_id': '10044', 'town': 'NEW YORK'}]


In [10]:
from bson.son import SON
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"city": 1, "_id": 1, "pop": 1}},
    # python only
    {"$sort": SON([("_id", 1),("pop", -1)])}
#    {"$sort": SON([("pop", -1), ("_id", 1)])}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:3])

[{'_id': '10001', 'city': 'NEW YORK', 'pop': 18913},
 {'_id': '10002', 'city': 'NEW YORK', 'pop': 84143},
 {'_id': '10003', 'city': 'NEW YORK', 'pop': 51224}]


In [11]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"city": 1, "_id": 1}},
    {"$limit": 2}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result))

[{'_id': '10001', 'city': 'NEW YORK'}, {'_id': '10002', 'city': 'NEW YORK'}]


In [12]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"city": 1, "_id": 1}},
    {"$skip": 18},
    {"$limit": 2},
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result))

[{'_id': '10022', 'city': 'NEW YORK'}, {'_id': '10023', 'city': 'NEW YORK'}]


### stage operators III

In [13]:
pipeline = [
    {"$group": {"_id": "$state",
               "totalPop": {"$sum": "$pop"}}},
    {"$match": {"totalPop": {"$gt": 10000000}}},
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result))

[{'_id': 'TX', 'totalPop': 16984601},
 {'_id': 'IL', 'totalPop': 11427576},
 {'_id': 'CA', 'totalPop': 29754890},
 {'_id': 'OH', 'totalPop': 10846517},
 {'_id': 'FL', 'totalPop': 12686644},
 {'_id': 'NY', 'totalPop': 17990402},
 {'_id': 'PA', 'totalPop': 11881643}]


In [14]:
"""
SELECT state, SUM(pop) AS totalPop
FROM zipcodes
GROUP BY state
HAVING totalPop >= (10000000)
"""

'\nSELECT state, SUM(pop) AS totalPop\nFROM zipcodes\nGROUP BY state\nHAVING totalPop >= (10000000)\n'

In [15]:
pipeline = [
    {"$group": {"_id": "$state",
               "totalPop": {"$sum": "$pop"}}},
    {"$group": {"_id": None,
               "avgPop": {"$avg": "$totalPop"}}}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result))

[{'_id': None, 'avgPop': 4870752.94117647}]


In [16]:
pipeline = [
    {"$group": {"_id": {"state": "$state", "city": "$city"},
               "totalCityPop": {"$sum": "$pop"}}},
    {"$group": {"_id": "$_id.state",
               "avgCityPop": {"$avg": "$totalCityPop"}}},
    {"$sort": {"avgCityPop": -1}}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:5])

[{'_id': 'DC', 'avgCityPop': 303450.0},
 {'_id': 'CA', 'avgCityPop': 27756.42723880597},
 {'_id': 'FL', 'avgCityPop': 27400.958963282937},
 {'_id': 'AZ', 'avgCityPop': 20591.16853932584},
 {'_id': 'RI', 'avgCityPop': 19292.653846153848}]


In [17]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$lookup": {
            "from": "zips",  # collection to join
            "localField": "city",  # field from the input documents
            "foreignField": "city",  # field from the documents of the "from" collection
            "as": "selfj"
        }} 
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:5])

[{'_id': '10001',
  'city': 'NEW YORK',
  'loc': [-73.996705, 40.74838],
  'pop': 18913,
  'selfj': [{'_id': '10001',
             'city': 'NEW YORK',
             'loc': [-73.996705, 40.74838],
             'pop': 18913,
             'state': 'NY'},
            {'_id': '10002',
             'city': 'NEW YORK',
             'loc': [-73.987681, 40.715231],
             'pop': 84143,
             'state': 'NY'},
            {'_id': '10003',
             'city': 'NEW YORK',
             'loc': [-73.989223, 40.731253],
             'pop': 51224,
             'state': 'NY'},
            {'_id': '10005',
             'city': 'NEW YORK',
             'loc': [-74.008344, 40.705649],
             'pop': 202,
             'state': 'NY'},
            {'_id': '10006',
             'city': 'NEW YORK',
             'loc': [-74.013474, 40.708451],
             'pop': 119,
             'state': 'NY'},
            {'_id': '10007',
             'city': 'NEW YORK',
             'loc': [-74.007022, 40.713